In [26]:
import polars as pl
import pandas as pd
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:

DATA_DIR = Path("data")

RETAILER_PATH = DATA_DIR / "retailer.csv"
TV_PATH = DATA_DIR / "tv_publisher.csv"
PROG_PATH = DATA_DIR / "programmatic_publisher.csv"
MAP_PATH = DATA_DIR / "mapping_transac_publisher_tv.csv"

OUT_PATH = DATA_DIR / "customer_events.parquet"

In [3]:
df_retailer = pd.read_csv(RETAILER_PATH)
df_tv_publisher = pd.read_csv(TV_PATH)
df_prog_publisher = pd.read_csv(PROG_PATH)
df_map = pd.read_csv(MAP_PATH)

# Cleaning retailer

In [4]:
df_retailer.shape

(9866049, 7)

In [5]:
df_retailer['customer_id'].value_counts()

customer_id
unknown               194530
reth7mdoDJCbkDalaK      6190
reJZhNiD0cXQyewGn3      5658
reHQRAI2c9b35Tij3O      2230
rejr2YF7XyHh2RavMT      2081
                       ...  
re4j5Kk86B1uunxSLJ         1
reJRLE6SzA8kzVVG4S         1
rerWAZSpigQx5UnZMZ         1
reBda2QsASNpnCCjNY         1
rey044Zf6he5jBVrVo         1
Name: count, Length: 1354584, dtype: int64

In [6]:
df_retailer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9866049 entries, 0 to 9866048
Data columns (total 7 columns):
 #   Column         Dtype  
---  ------         -----  
 0   customer_id    object 
 1   timestamp_utc  object 
 2   event_name     object 
 3   brand          object 
 4   product_name   object 
 5   sales          float64
 6   quantity       float64
dtypes: float64(2), object(5)
memory usage: 526.9+ MB


In [7]:
df_retailer["timestamp_utc"] = pd.to_datetime(df_retailer["timestamp_utc"], errors="coerce")


In [9]:
df_retailer['timestamp_utc'].min(), df_retailer['timestamp_utc'].max()

(Timestamp('2024-01-01 00:00:01'), Timestamp('2024-06-30 23:59:59'))

In [12]:
df_retailer["customer_id"] = df_retailer["customer_id"].astype("category")
df_retailer["event_name"]  = df_retailer["event_name"].astype("category")

df_retailer["sales"] = pd.to_numeric(df_retailer["sales"], errors="coerce").astype("float32")

In [13]:
df_retailer = df_retailer[df_retailer["customer_id"] != "unknown"]


In [16]:
print("rows:", len(df_retailer))
print("unique customers:", df_retailer["customer_id"].nunique())
print("memory (GB):", df_retailer.memory_usage(deep=True).sum() / 1e9)

rows: 9671519
unique customers: 1354583
memory (GB): 1.770859527


In [24]:
df_retailer["day"] = df_retailer["timestamp_utc"].dt.floor("D")

customer_agg = (
    df_retailer
    .groupby("customer_id", sort=False)
    .agg(
        nb_events=("customer_id", "size"),
        nb_add_to_cart=("event_name", lambda x: (x == "Add to cart").sum()),
        nb_order=("event_name", lambda x: (x == "Order").sum()),
        nb_view=("event_name", lambda x: (x == "Product Page View").sum()), 
        total_sales=("sales", "sum"),
        total_quantity=("quantity", "sum"),
        active_days=("day", "nunique"),
        first_event=("timestamp_utc", "min"),
        last_event=("timestamp_utc", "max"),
    )
    .reset_index()
)


C:\Users\adamc\AppData\Local\Temp\ipykernel_17216\2670320693.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby("customer_id", sort=False)


In [30]:
customer_agg.describe()

,nb_events,nb_add_to_cart,nb_order,nb_view,total_sales,total_quantity,active_days,first_event,last_event,events_per_day,orders_per_day,avg_basket
count,1.354584e+06,1.354584e+06,1.354584e+06,1.354584e+06,1.354584e+06,1.354584e+06,1.354584e+06,1354583,1354583,1.354583e+06,1.354583e+06,502025.000000
mean,7.139844e+00,1.483650e+00,1.041566e+00,4.614629e+00,4.517511e+01,1.125864e+00,2.434611e+00,2024-03-09 03:25:08.520636416,2024-04-11 23:04:54.294809856,2.588428e+00,2.812881e-01,44.559067
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2024-01-01 00:00:04,2024-01-01 00:00:50,1.000000e+00,0.000000e+00,0.000000
25%,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2024-01-23 22:22:56,2024-02-21 14:55:57.500000,1.000000e+00,0.000000e+00,22.490000
50%,3.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2024-02-24 04:08:00,2024-04-18 20:37:39,2.000000e+00,0.000000e+00,41.298001
75%,8.000000e+00,2.000000e+00,1.000000e+00,5.000000e+00,4.788000e+01,1.000000e+00,3.000000e+00,2024-04-20 16:28:28,2024-06-02 20:38:44,3.083333e+00,5.000000e-01,57.820002
max,6.190000e+03,2.051000e+03,2.013000e+03,2.999000e+03,4.757174e+04,2.120000e+03,1.770000e+02,2024-06-30 23:59:34,2024-06-30 23:59:59,1.365000e+02,1.447126e+01,1184.699951
std,1.393931e+01,3.593188e+00,3.033672e+00,9.317185e+00,1.178023e+02,3.436847e+00,2.760003e+00,NaN,NaN,2.247819e+00,4.300321e-01,26.237011


In [27]:
customer_agg["events_per_day"] = (
    customer_agg["nb_events"] / customer_agg["active_days"].replace(0, np.nan)
)

customer_agg["orders_per_day"] = (
    customer_agg["nb_order"] / customer_agg["active_days"].replace(0, np.nan)
)

customer_agg["avg_basket"] = (
    customer_agg["total_sales"] / customer_agg["nb_order"].replace(0, np.nan)
)


In [29]:
metrics = [
    "nb_events",
    "nb_view",
    "nb_add_to_cart",
    "nb_order",
    "events_per_day",
    "orders_per_day",
    "total_sales",
    "total_quantity",
    "avg_basket"
]

customer_agg[metrics].quantile([0.95, 0.99, 0.999])


,nb_events,nb_view,nb_add_to_cart,nb_order,events_per_day,orders_per_day,total_sales,total_quantity,avg_basket
0.950,26.0,16.0,7.0,5.0,6.5,1.000000,228.368496,5.0,84.839996
0.990,52.0,35.0,13.0,10.0,11.0,1.500000,485.761711,12.0,125.989998
0.999,114.0,88.0,27.0,21.0,21.0,2.666667,1074.405960,28.0,203.979996


There seems to be no clear outliers here, dropping "unknown" seems to be sufficient.

# Cleaning TV

In [31]:
df_tv_publisher["timestamp_utc"] = pd.to_datetime(df_tv_publisher["timestamp_utc"], errors="coerce", utc=True)
df_tv_publisher["cost_milli_cent"] = pd.to_numeric(df_tv_publisher["cost_milli_cent"], errors="coerce")

In [36]:
df_tv_publisher = df_tv_publisher[df_tv_publisher["device_id"] != "unknown"]

In [37]:
device_counts = (
    df_tv_publisher
    .groupby("device_id")
    .size()
    .rename("nb_exposures")
    .reset_index()
)


In [38]:
device_counts.describe()

,nb_exposures
count,2.065538e+06
mean,2.737266e+00
std,4.337041e+00
min,1.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,3.000000e+00
max,2.340000e+02


Again, the values seem acceptable, we only drop unknown.

# Cleaning Programmatic_publisher

In [45]:
df_prog_publisher["dsp_id"].value_counts()

dsp_id
unknown                           1907730
dsphWn1b3sH6wluDvi3ipsvHSqSs1A        194
dspGgGMy9D8tTvo7IoHwXOaUOYNlKE        193
dspfhqXaeTQf24O3RE7uaAakbcMaS3        171
dspK9A148mt6dpOvdrmDxPaOE7VWX6        163
                                   ...   
dspAdgifdTCnmyt0CKqcJLjKsVDxja          1
dspIxuJl7MJCmDflz6H7hmCyZYHNVP          1
dspIlnXTcp4L18ciFtAWZjr82hu4kG          1
dspP9Qbar5GwBSTnKyNLW8Y6A2wzHO          1
dspBO95y6pH7zjqT8ZFEHUm86hZ9eC          1
Name: count, Length: 5101025, dtype: int64

In [46]:
df_prog_publisher = df_prog_publisher[df_prog_publisher['dsp_id'] != 'unknown']

In [47]:
df_prog_publisher['device_type'].value_counts()

device_type
PC         9102841
Phone      5280813
TV         1201410
Unknown        623
Robot           11
Name: count, dtype: int64

In [48]:
df_prog = df_prog_publisher.copy()

df_prog["timestamp_utc"] = pd.to_datetime(
    df_prog["timestamp_utc"], errors="coerce", utc=True
)
df_prog["cost_milli_cent"] = pd.to_numeric(
    df_prog["cost_milli_cent"], errors="coerce"
)


In [49]:
device_stats = (
    df_prog
    .groupby("device_type")
    .agg(
        nb_events=("dsp_id", "count"),
        nb_users=("dsp_id", "nunique"),
        avg_cost=("cost_milli_cent", "mean"),
        median_cost=("cost_milli_cent", "median"),
        max_cost=("cost_milli_cent", "max"),
        first_event=("timestamp_utc", "min"),
        last_event=("timestamp_utc", "max"),
    )
    .assign(
        events_per_user=lambda df: df["nb_events"] / df["nb_users"]
    )
)

device_stats.sort_values("events_per_user", ascending=False)


,nb_events,nb_users,avg_cost,median_cost,max_cost,first_event,last_event,events_per_user
device_type,,,,,,,,
PC,9102841,3393480,542.645341,229.000,10328.330,2024-01-01 00:00:04+00:00,2024-06-30 23:59:58+00:00,2.682450
Phone,5280813,2044977,379.838457,271.699,8526.250,2024-01-01 00:13:18+00:00,2024-06-30 23:59:59+00:00,2.582334
TV,1201410,689247,1112.680053,315.944,11335.000,2024-01-01 00:05:08+00:00,2024-06-30 23:59:43+00:00,1.743076
Robot,11,8,305.910364,223.790,914.228,2024-01-01 18:59:10+00:00,2024-04-16 04:54:39+00:00,1.375000
Unknown,623,462,596.536435,264.051,5506.967,2024-01-01 05:02:44+00:00,2024-06-26 19:36:44+00:00,1.348485


"Robots" and "unknown" seem here to have normal caracteristics, we choose to keep them.

# Mapping and merging datasets.

In [50]:
mapping = df_map.copy()

In [51]:
mapping.isna().mean()


customer_id    0.000000
dsp_id         0.000000
device_id      0.019876
dtype: float64

In [52]:
mapping.nunique()


customer_id    7825711
dsp_id         7115101
device_id      7593089
dtype: int64

In [53]:
mapping.groupby("device_id")["customer_id"].nunique().describe()


count    7.593089e+06
mean     1.030636e+00
std      8.441890e+01
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      2.326220e+05
Name: customer_id, dtype: float64

In [54]:
mapping.groupby("dsp_id")["customer_id"].nunique().describe()


count    7.115101e+06
mean     1.119720e+00
std      3.193429e+02
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      8.518210e+05
Name: customer_id, dtype: float64

We can see here some outliers.
We will exclude Medias ID associated with an abnormally high number of customers in order to avoid attribution errors and artificial customer journeys.

In [55]:
device_customer_count = (
    mapping.groupby("device_id")["customer_id"]
    .nunique()
    .reset_index(name="nb_customers")
)

valid_devices = device_customer_count.loc[
    device_customer_count["nb_customers"] <= 5,
    "device_id"
]

In [56]:
dsp_customer_count = (
    mapping.groupby("dsp_id")["customer_id"]
    .nunique()
    .reset_index(name="nb_customers")
)

valid_dsp = dsp_customer_count.loc[
    dsp_customer_count["nb_customers"] <= 5,
    "dsp_id"
]


In [57]:
mapping_clean = mapping[
    mapping["device_id"].isin(valid_devices) &
    mapping["dsp_id"].isin(valid_dsp)
].copy()


In [ ]:
#Mapper TV → customer_id
df_tv_mapped = (
    df_tv_publisher
    .merge(
        mapping_clean[["customer_id", "device_id"]],
        on="device_id",
        how="inner"
    )
)

df_tv_mapped["channel"] = "TV"
df_tv_mapped["event_type"] = "exposure"
df_tv_mapped["campaign_name"] = None
df_tv_mapped["device_type"] = "TV"


In [ ]:
#Mapper Programmatic → customer_id
df_prog_mapped = (
    df_prog_publisher
    .merge(
        mapping_clean[["customer_id", "dsp_id"]],
        on="dsp_id",
        how="inner"
    )
)

df_prog_mapped["channel"] = "PROGRAMMATIC"
df_prog_mapped["event_type"] = "exposure"


In [62]:
#Uniform column names

tv_events = df_tv_mapped[[
    "customer_id",
    "timestamp_utc",
    "channel",
    "event_type",
    "campaign_name",
    "device_type",
    "cost_milli_cent"
]].rename(columns={"cost_milli_cent": "cost"})


In [63]:
prog_events = df_prog_mapped[[
    "customer_id",
    "timestamp_utc",
    "channel",
    "event_type",
    "campaign_name",
    "device_type",
    "cost_milli_cent"
]].rename(columns={"cost_milli_cent": "cost"})


In [64]:
media_events = pd.concat(
    [tv_events, prog_events],
    ignore_index=True
)


In [65]:
retail_events = df_retailer[[
    "customer_id",
    "timestamp_utc",
    "event_name",
    "sales",
    "quantity"
]].copy()

retail_events["channel"] = "RETAIL"
retail_events["event_type"] = retail_events["event_name"]
retail_events["campaign_name"] = None
retail_events["device_type"] = None
retail_events["cost"] = None


In [66]:
customer_journey = pd.concat(
    [
        media_events[[
            "customer_id", "timestamp_utc", "channel",
            "event_type", "campaign_name", "device_type", "cost"
        ]],
        retail_events[[
            "customer_id", "timestamp_utc", "channel",
            "event_type", "campaign_name", "device_type",
            "cost", "sales", "quantity"
        ]]
    ],
    ignore_index=True,
    sort=False
)


C:\Users\adamc\AppData\Local\Temp\ipykernel_17216\2060446389.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  customer_journey = pd.concat(


In [71]:
customer_journey["timestamp_utc"] = pd.to_datetime(
    customer_journey["timestamp_utc"],
    errors="coerce",
    utc=True
)


In [72]:
#Sorting by customer_id and timestamp
customer_journey = customer_journey.sort_values(
    ["customer_id", "timestamp_utc"]
)

In [68]:
customer_journey["channel"].value_counts(normalize=True)


channel
PROGRAMMATIC    0.503610
RETAIL          0.328602
TV              0.167788
Name: proportion, dtype: float64

In [69]:
customer_journey.shape

(29432311, 9)

In [70]:
customer_journey.isna().mean().sort_values(ascending=False)


sales            0.952063
quantity         0.952063
campaign_name    0.496390
device_type      0.328602
cost             0.328602
customer_id      0.000000
event_type       0.000000
timestamp_utc    0.000000
channel          0.000000
dtype: float64

In [73]:
customer_journey.head()

,customer_id,timestamp_utc,channel,event_type,campaign_name,device_type,cost,sales,quantity
3798507,re0000ojYmqRl7rUnS,2024-03-08 00:04:01+00:00,TV,exposure,None,TV,2325.51,NaN,NaN
3798508,re0000ojYmqRl7rUnS,2024-03-21 01:06:25+00:00,TV,exposure,None,TV,2325.51,NaN,NaN
3798509,re0000ojYmqRl7rUnS,2024-03-21 02:17:34+00:00,TV,exposure,None,TV,2325.51,NaN,NaN
3798510,re0000ojYmqRl7rUnS,2024-03-22 10:43:36+00:00,TV,exposure,None,TV,2325.51,NaN,NaN
3798511,re0000ojYmqRl7rUnS,2024-03-25 02:32:02+00:00,TV,exposure,None,TV,2325.51,NaN,NaN


In [74]:
customer_journey.to_parquet(
    "customer_journey.parquet",
    index=False
)
